<a href="https://colab.research.google.com/github/HisakaKoji/COVID-19/blob/master/%E3%80%90PyTorch%E3%80%91Transformer%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E6%97%A5%E6%9C%AC%E8%AA%9E%E6%96%87%E7%AB%A0%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://qiita.com/yamaru/items/dea6b8076baefe38905d

In [1]:
!pip install PyDrive
!pip install janome
!pip install mojimoji

     |████████████████████████████████| 21.5MB 1.5MB/s 
     |████████████████████████████████| 133kB 2.8MB/s 


左のドライブマウントを有効にしてください。


In [4]:
!git clone --branch master --depth 1 https://github.com/aozorabunko/aozorabunko.git "drive/My Drive/transformert/20200516test"

Cloning into 'drive/My Drive/transformert/20200516test'...
remote: Enumerating objects: 69677, done.
remote: Counting objects: 100% (69677/69677), done.
remote: Compressing objects: 100% (55892/55892), done.
remote: Total 69677 (delta 22989), reused 50606 (delta 13598), pack-reused 0
Receiving objects: 100% (69677/69677), 2.04 GiB | 14.41 MiB/s, done.
Resolving deltas: 100% (22989/22989), done.
Checking out files: 100% (67709/67709), done.


In [0]:
#----------------------
# 対象ファイル一覧の取得
#----------------------
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

# Google driveへのアクセスを許可
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 全作品のタイトルとdrive上のidを取得
def get_list_file_recursively(parent_id, l=None):
    if l is None:
        l = []

    file_list = drive.ListFile({'q': '"{}" in parents and trashed = false'.format(parent_id)}).GetList()
    l += [f for f in file_list if f['mimeType'] != 'application/vnd.google-apps.folder']

    for f in file_list:
        if f['mimeType'] == 'application/vnd.google-apps.folder':
            get_list_file_recursively(f['id'], l)

    return l

listed = []
#google drive のID　　
for f in get_list_file_recursively('1--4eS7otRWMzPOafG4qXbutZlEEeK0mY'):
  print(f['title'])
  if 'html' in f['title'] and 'card' not in f['title']:
    list = [f['title'], f['id']]
    listed.append(list)
listed = pd.DataFrame(listed)


In [16]:
listed

,0,1
0,aozora_bunkono_shikumi.html,11GFhUjQFLjJ2pfWLmSyAuoBHjIbXGovp
1,aozoraML.html,11ISHdyAyWF0qzA6RImXRJmy9rslFdJ19
2,accent_separation.html,11Wn5n8HbcjWh9TcFtcywRmjkA95gCKYe
3,DOKUSYO.html,1180XFJAHlShxGa3rNpsWNd5ay7gIcilD
4,46844_26169.html,1LzWF5C0Zr0f7oBd7cHe4L7s4t_bMUEis
...,...,...
6025,MANUAL_2.html,16ZzdJIQaUemOOHVKVWn7b2dnD39dpySk
6026,MANUAL_1.html,16_yRC8Sn3thtxaTiFigskyO08eE4AYNf
6027,index.html,1GFVDcs1jSIKlFIyJ7e49cW_cP8rls06X
6028,guide_line.html,1KpkpmqBlgSjR5kIkltUqABOyJbKhYPcs


In [0]:
 file_data = drive.CreateFile({'id': listed.iloc[0, 0]})

In [23]:
file_data

GoogleDriveFile({'id': 'aozora_bunkono_shikumi.html'})

In [21]:
file_data

GoogleDriveFile({'id': '1HHc9WETS1-93bnxQI63KYuDxBRQdbio5', 'kind': 'drive#file', 'etag': '"MTU4OTYwNDUyMzIwOA"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1HHc9WETS1-93bnxQI63KYuDxBRQdbio5', 'webContentLink': 'https://drive.google.com/uc?id=1HHc9WETS1-93bnxQI63KYuDxBRQdbio5&export=download', 'alternateLink': 'https://drive.google.com/file/d/1HHc9WETS1-93bnxQI63KYuDxBRQdbio5/view?usp=drivesdk', 'embedLink': 'https://drive.google.com/file/d/1HHc9WETS1-93bnxQI63KYuDxBRQdbio5/preview?usp=drivesdk', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/text/xml', 'title': '46393_24242.html', 'mimeType': 'text/xml', 'labels': {'starred': False, 'hidden': False, 'trashed': False, 'restricted': False, 'viewed': True}, 'copyRequiresWriterPermission': False, 'createdDate': '2020-05-16T04:48:43.208Z', 'modifiedDate': '2020-05-16T04:48:43.208Z', 'modifiedByMeDate': '2020-05-16T04:48:43.208Z', 'lastViewedByMeDate': '2020-05-16T05:18:54.416Z', 'markedViewedByMeDate': '197

時間かかります。！！

In [29]:

#----------------------
# タイトル・本文の取得
#----------------------
from bs4 import BeautifulSoup

# リストのhtmlファイルを読込
Stories = []
print(len(listed))
for i in range(0, len(listed)):
    if i % 100 == 0:
        print('{} / {}'.format(i, len(listed)))

    # リストのidからファイルを特定
    file_data = drive.CreateFile({'id': listed.iloc[i, 1]})
    file_data.GetContentFile(listed.iloc[i, 0])
    print(listed.iloc[i, 0])
    with open(listed.iloc[i, 0], 'rb') as html:
        soup = BeautifulSoup(html, 'lxml')

    # タイトル・本文を取得
    title = soup.find("h1", class_='title')

    main_text = soup.find("div", class_='main_text')
    #print(main_text)
    # タイトルか本文が欠損の場合は飛ばす
    if title == None or main_text == None:
        continue

    # ルビを削除
    for yomigana in main_text.find_all(["rp", "h4", "rt"]):
        yomigana.decompose()

    # 整形して文字列化
    title = [line.strip() for line in 
    title.text.strip().splitlines()]
    main_text = [line.strip() for line in 
    main_text.text.strip().splitlines()]
    title = ''.join(title)
    text=''.join(main_text)
    print(title) 
    # 本文が3,000文字以内の作品に絞込み
    if len(text) <= 3000:
        Stories.append([title, text])



6030
0 / 6030
aozora_bunkono_shikumi.html
青空文庫のしくみ
aozoraML.html
accent_separation.html
DOKUSYO.html
46844_26169.html
幕末維新懐古談
46843_26168.html
幕末維新懐古談
46842_26167.html
幕末維新懐古談
46841_26166.html
幕末維新懐古談
46840_26165.html
幕末維新懐古談
46637_25176.html
幕末維新懐古談
46396_24245.html
幕末維新懐古談
46395_24244.html
幕末維新懐古談
46394_24243.html
幕末維新懐古談
46393_24242.html
幕末維新懐古談
46203_24241.html
幕末維新懐古談
46202_24240.html
幕末維新懐古談
46201_24239.html
幕末維新懐古談
46200_24238.html
幕末維新懐古談
45964_21737.html
幕末維新懐古談
1552_21475.html
幕末維新懐古談
1551_21474.html
幕末維新懐古談
1550_21473.html
幕末維新懐古談
1549_21472.html
幕末維新懐古談
1548_21471.html
幕末維新懐古談
1547_21470.html
幕末維新懐古談
1546_21469.html
幕末維新懐古談
1545_18900.html
佐竹の原へ大仏をこしらえたはなし
1543_24303.html
般若心経講義
46287_22314.html
手数将棋
1542_20955.html
本因坊と私
1541_22312.html
駒台の発案者
59421_70485.html
女子霧ヶ峰登山記
46174_24886.html
諏訪湖畔冬の生活
42773_39853.html
ロミオとヂュリエット
53817_53604.html
東奥異聞
56519_61215.html
国際射的大競技
56330_61354.html
ジェンナー伝
54116_68236.html
五階の窓
48364_31535.html
江戸川氏と私
48086_38520.html
ある自殺者の手記
48085_3851

InvalidConfigError: ignored

In [0]:
# csvとして保存
Stories = pd.DataFrame(Stories)
Stories.to_csv('drive/My Drive/Stories.csv', index=False, header=False)

In [0]:
Stories[0]

In [0]:
a= kf.split(Stories)

In [0]:

print (result)

In [0]:
#----------------------
# データの分割
#----------------------
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=12345)
result = next(kf.split(Stories), None)

train = Stories.iloc[result[0]]
test =  Stories.iloc[result[1]]
#train = df.iloc[tr_idx, :]
#test = df.iloc[te_idx, :]
train.to_csv('drive/My Drive/train.csv', index=False, header=False)
test.to_csv('drive/My Drive/test.csv', index=False, header=False)

In [0]:
#----------------------
# 前処理の定義
#----------------------
from torchtext import data
from janome.tokenizer import Tokenizer
import re
import mojimoji

# 文字列処理定義
def preprocessing(text):
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)
    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]', '0', text) 
    # 全角化
    text = mojimoji.han_to_zen(text)
    return text

# Tokenizer定義
j_t = Tokenizer()
def tokenizer(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]

# 文字列処理 + Tokenizer
def tokenizer_with_preprocessing(text):
    text = preprocessing(text)
    text = tokenizer(text)
    return text

In [0]:
#----------------------
# Fieldの定義
#----------------------
TEXT = data.Field(
    sequential=True, 
    init_token='<sos>', 
    eos_token='<eos>', 
    tokenize=tokenizer_with_preprocessing, 
    lower=True, 
    use_vocab=True, 
    batch_first=True
)

In [0]:
#----------------------
# データの読込
#----------------------
train_ds, test_ds = data.TabularDataset.splits(
    path='drive/My Drive',
    train='train.csv',
    test='test.csv',
    format='csv',
    skip_header=False,
    fields=[('title', TEXT), ('text', TEXT)]
)

# 確認
train_ds[0].__dict__.keys()
test_ds[0].__dict__.keys()
for i in range(0, 10):
    print(vars(train_ds[i]))
    print(vars(test_ds[i]))

# 辞書作成
TEXT.build_vocab(train_ds, test_ds, min_freq=2)

# 単語カウント
print(TEXT.vocab.freqs)
print('語彙数:{}'.format(len(TEXT.vocab)))



In [49]:
!pip install transformers==2.9.0

     |████████████████████████████████| 645kB 2.6MB/s 
     |████████████████████████████████| 1.1MB 9.5MB/s 
     |████████████████████████████████| 890kB 19.1MB/s 
     |████████████████████████████████| 3.8MB 24.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ae0e0a2ce1200ddc1f435d935dd08078affa7e696e33c20c54eeea5ec22294d3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [69]:
import torch
# イテレータの作成

device ='cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# --> 事前に「ランタイムのタイプを変更」からGPUを選択しておく。
train_iter = data.Iterator(train_ds, batch_size=16, shuffle=True, device=device)
test_iter = data.Iterator(test_ds, batch_size=16, shuffle=False, device=device)

# 確認
batch = next(iter(train_iter))
print(batch.title)
print(batch.text)

batch = next(iter(test_iter))
print(batch.title)
print(batch.text)

tensor([[    2,  1375, 14036,     0,  1376,     3,     1,     1,     1,     1],
        [    2,   260,     5,  6790,     3,     1,     1,     1,     1,     1],
        [    2, 15261,   182,     3,     1,     1,     1,     1,     1,     1],
        [    2,  2045,     5,    57,     3,     1,     1,     1,     1,     1],
        [    2,  5069,  3506,   451,     3,     1,     1,     1,     1,     1],
        [    2,   469,  2005,  3295,     5,  6779,     3,     1,     1,     1],
        [    2,  1130,    13,    27,   444,     3,     1,     1,     1,     1],
        [    2,   486,  3790,     3,     1,     1,     1,     1,     1,     1],
        [    2,   986,     7,  9092,    10,     3,     1,     1,     1,     1],
        [    2,   278,  2676,     5,   312,     3,     1,     1,     1,     1],
        [    2,  2989,  3179,     3,     1,     1,     1,     1,     1,     1],
        [    2,  2803,     9,     0,   129,     3,     1,     1,     1,     1],
        [    2,  2953,     5,  7961,    

In [61]:
pip install pytorch_memlab

  Created wheel for pytorch-memlab: filename=pytorch_memlab-0.1.0-cp36-none-any.whl size=13458 sha256=eb267e6d5a66214c979db0d15ff3825b6fc12362d46b386b1eaf6bf6a0751952
  Stored in directory: /root/.cache/pip/wheels/25/84/ed/12ba6ba64bc673b705f987a54743419592afdc081e0d546359
  Created wheel for calmsize: filename=calmsize-0.1.3-cp36-none-any.whl size=2887 sha256=22569ba7a92a841226bcb268c940da0cf7299ff2da577797acbd90a1c2fb4aaf
  Stored in directory: /root/.cache/pip/wheels/02/ac/1a/2a3418bc8f81abda6cd1b6d0f8f287556fc8de80dbabab75d1
Successfully built pytorch-memlab calmsize


In [0]:
from pytorch_memlab import profile

class Net(nn.Module):
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    return x 

In [65]:
net = Net().cuda()
reporter = MemReporter(net)

reporter.report()

NameError: ignored

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
import torch
from torch import nn

class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length=100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)

        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])

        self.dropout = nn.Dropout(dropout)

        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

    def forward(self, src, src_mask):

        #src = [batch size, src len]
        #src_mask = [batch size, src len]

        batch_size = src.shape[0]
        src_len = src.shape[1]

        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        #pos = [batch size, src len]

        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))

        #src = [batch size, src len, hid dim]

        for layer in self.layers:
            src = layer(src, src_mask)

        #src = [batch size, src len, hid dim]

        return src


class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()

        self.layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask):

        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, src len]

        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)

        #dropout, residual connection and layer norm
        src = self.layer_norm(src + self.dropout(_src))

        #src = [batch size, src len, hid dim]

        #positionwise feedforward
        _src = self.positionwise_feedforward(src)

        #dropout, residual and layer norm
        src = self.layer_norm(src + self.dropout(_src))

        #src = [batch size, src len, hid dim]

        return src


class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()

        assert hid_dim % n_heads == 0

        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads

        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)

        self.fc_o = nn.Linear(hid_dim, hid_dim)

        self.dropout = nn.Dropout(dropout)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    def forward(self, query, key, value, mask = None):

        batch_size = query.shape[0]

        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]

        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]

        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        #energy = [batch size, n heads, seq len, seq len]

        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        attention = torch.softmax(energy, dim = -1)

        #attention = [batch size, n heads, query len, key len]

        x = torch.matmul(self.dropout(attention), V)

        #x = [batch size, n heads, seq len, head dim]

        x = x.permute(0, 2, 1, 3).contiguous()

        #x = [batch size, seq len, n heads, head dim]

        x = x.view(batch_size, -1, self.hid_dim)

        #x = [batch size, seq len, hid dim]

        x = self.fc_o(x)

        #x = [batch size, seq len, hid dim]

        return x, attention


class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()

        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        #x = [batch size, seq len, hid dim]

        x = self.dropout(torch.relu(self.fc_1(x)))

        #x = [batch size, seq len, pf dim]

        x = self.fc_2(x)

        #x = [batch size, seq len, hid dim]

        return x

In [0]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length=1000):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)

        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

    def forward(self, trg, enc_src, trg_mask, src_mask):

        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, trg len]
        #src_mask = [batch size, src len]

        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        #pos = [batch size, trg len]

        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))

        #trg = [batch size, trg len, hid dim]

        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)

        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]

        output = self.fc_out(trg)

        #output = [batch size, trg len, output dim]

        return output, attention


class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()

        self.layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, enc_src, trg_mask, src_mask):

        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, trg len]
        #src_mask = [batch size, src len]

        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)

        #dropout, residual connection and layer norm
        trg = self.layer_norm(trg + self.dropout(_trg))

        #trg = [batch size, trg len, hid dim]

        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)

        #dropout, residual connection and layer norm
        trg = self.layer_norm(trg + self.dropout(_trg))

        #trg = [batch size, trg len, hid dim]

        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)

        #dropout, residual and layer norm
        trg = self.layer_norm(trg + self.dropout(_trg))

        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]

        return trg, attention

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):

        #src = [batch size, src len]

        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask

    def make_trg_mask(self, trg):

        #trg = [batch size, trg len]

        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(3)

        #trg_pad_mask = [batch size, 1, trg len, 1]

        trg_len = trg.shape[1]

        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()

        #trg_sub_mask = [trg len, trg len]

        trg_mask = trg_pad_mask & trg_sub_mask

        #trg_mask = [batch size, 1, trg len, trg len]

        return trg_mask

    def forward(self, src, trg):

        #src = [batch size, src len]
        #trg = [batch size, trg len]

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]

        enc_src = self.encoder(src, src_mask)

        #enc_src = [batch size, src len, hid dim]

        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]

        return output, attention

In [78]:
!watch -n1 "nvidia-smi -i 1"

>

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):

        #src = [batch size, src len]

        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask

    def make_trg_mask(self, trg):

        #trg = [batch size, trg len]

        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(3)

        #trg_pad_mask = [batch size, 1, trg len, 1]

        trg_len = trg.shape[1]

        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()

        #trg_sub_mask = [trg len, trg len]

        trg_mask = trg_pad_mask & trg_sub_mask

        #trg_mask = [batch size, 1, trg len, trg len]

        return trg_mask

    def forward(self, src, trg):

        #src = [batch size, src len]
        #trg = [batch size, trg len]

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]

        enc_src = self.encoder(src, src_mask)

        #enc_src = [batch size, src len, hid dim]

        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]

        return output, attention

In [76]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=58490412b8f336027a5ae6a2235834e4793e056b7bb756633706d9dfb11666cf
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 10.6 GB  | Proc size: 2.9 GB
GPU RAM Free: 439MB | Used: 15841MB | Util  97% | Total 16280MB


In [0]:
#----------------------
# 学習準備
#----------------------
# パラメータの設定
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(TEXT.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

# Encoderの初期化
enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

# Decoderの初期化
dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

# padding用のIDの指定
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

# モデルの初期化
model = Seq2Seq(enc, dec, PAD_IDX, PAD_IDX, device).to(device)

# 重みの初期化
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)
model.apply(initialize_weights)

# オプティマイザーの設定
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 損失関数の設定
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# 学習用関数の定義
def train(model, iterator, optimizer, criterion, clip):

    model.train()

    epoch_loss = 0

    for i, batch in enumerate(iterator):

        src = batch.title
        trg = batch.text

        optimizer.zero_grad()

        output, _ = model(src, trg[:,:-1])

        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]

        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)

        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

# 評価用関数の定義
def evaluate(model, iterator, criterion):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for i, batch in enumerate(iterator):

            src = batch.title
            trg = batch.text

            output, _ = model(src, trg[:,:-1])

            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)

            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

# 処理時間測定用関数の定義
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# 文章生成用関数の定義
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=1000):

    model.eval()

    tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    src_mask = model.make_src_mask(src_tensor)

    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)

        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)

        pred_token = output.argmax(2)[:,-1].item()

        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    return trg_tokens[1:], attention


#----------------------
# モデルの学習
#----------------------
import time
import math

N_EPOCHS = 100
CLIP = 1

# サンプル1作品を取得
example_idx = 8
src_sample = vars(train_ds.examples[example_idx])['title']
trg_sample = vars(train_ds.examples[example_idx])['text']

# タイトルと本文を表示
print(f'src = {src_sample}')
print(f'trg = {trg_sample}')

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    #if valid_loss < best_valid_loss:
    #    best_valid_loss = valid_loss
    #    torch.save(model.state_dict(), 'drive/My Drive/trained_model.pt')
    # 参考スクリプトではバリデーションデータの精度を指標にしているが、今回の試行では学習が進むほどバリデーションデータの精度が低下したため、オーバーフィットは無視して最終エポック後のモデルを採用（途中終了も考慮して各エポック後に保存）
    torch.save(model.state_dict(), 'drive/My Drive/trained_model.pt')

    # エポックごとに学習用・バリデーション用データの精度を表示
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

    # 10エポックごとにサンプル1作品のタイトルから本文を生成した結果を表示
    if epoch % 10 == 0:
      translation, attention = translate_sentence(src_sample, TEXT, TEXT, model, device)
      print(f'predicted trg = {translation}')

In [0]:
translation, attention = translate_sentence([''], TEXT, TEXT, model, device)
print(f'predicted trg = {translation}')